In [47]:
import os
import netCDF4 as nc4
import numpy as np


data_path = 'c:/4Hackathon/Data/OLCI Level 1 Level 2/S3A_OL_1_EFR____20161120T112457_20161120T112739_20161120T135747_0161_011_137_1998_MAR_O_NR_002/S3A_OL_1_EFR____20161120T112457_20161120T112739_20161120T135747_0161_011_137_1998_MAR_O_NR_002.SEN3/'
rootgrp1 = nc4.Dataset(data_path+"geo_coordinates.nc")
#rootgrp1.variables.keys()
lat = rootgrp1.variables['latitude'][:]
lon = rootgrp1.variables['longitude'][:]

#rootgrp2 = nc4.Dataset(data_path+"time_coordinates.nc")
#rootgrp1.variables.keys()
#timestamp = rootgrp2.variables['time_stamp'][:]





In [53]:
print(lat.max())
print(lat.min())

61.779856
49.841704


In [44]:
from io import StringIO
import psycopg2
cpy = StringIO()

for latindex, latitued in enumerate(lat):

    a = np.hstack((
              lat[latindex].reshape(lat[latindex].size,1),
              lon[latindex].reshape(lon[latindex].size,1)
    ))

    # Fill the cStringIO with text representation of the created array
    for row in a:
            cpy.write('\t'.join([str(x) for x in row[0:]]) + '\n')


conn = psycopg2.connect("host=localhost dbname=test user=postgres password=test123")
curs = conn.cursor()

cpy.seek(0)
curs.copy_from(cpy, 'ncdata', columns=('lat', 'lon'))
conn.commit()

In [41]:
rootgrp2.variables["time_stamp"].units
dates = num2date(rootgrp2.variables['time_stamp'][:],
                 rootgrp2.variables['time_stamp'].units)

In [ ]:
dates = num2date(rootgrp.variables['time'][:],units=rootgrp.variables['time'].units)
var1=rootgrp.variables['var1']
var2=rootgrp.variables['var2']

cpy = cStringIO.StringIO()

for timeindex, time in enumerate(dates):

    validtimes=np.empty(var1[timeindex].size, dtype="object")
    validtimes.fill(time)

    #  Transponse and stack the arrays of parameters
    #    [a,a,a,a]        [[a,b,c],
    #    [b,b,b,b]  =>     [a,b,c],
    #    [c,c,c,c]         [a,b,c],
    #                      [a,b,c]]

    a = np.hstack((
              validtimes.reshape(validtimes.size,1),
              stationnames.reshape(stationnames.size,1),
              var1[timeindex].reshape(var1[timeindex].size,1),
              var2[timeindex].reshape(var2[timeindex].size,1)
    ))

    # Fill the cStringIO with text representation of the created array
    for row in a:
            cpy.write(row[0].strftime("%Y-%m-%d %H:%M")+'\t'+ row[1] +'\t' + '\t'.join([str(x) for x in row[2:]]) + '\n')


conn = psycopg2.connect("host=postgresserver dbname=nc user=user password=passwd")
curs = conn.cursor()

cpy.seek(0)
curs.copy_from(cpy, 'ncdata', columns=('validtime', 'stationname', 'var1', 'var2'))
conn.commit()